In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import random
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, f1_score

# ─── GPU SETUP ───────────────────────────────────────────────────────────────
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
for gpu in tf.config.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)

# ─── REPRODUCIBILITY ─────────────────────────────────────────────────────────
def set_random_seeds(seed=42):
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
set_random_seeds(42)

# ─── LEARNING RATE SCHEDULE ───────────────────────────────────────────────────
def lr_schedule(epoch, initial_lr=3e-4, warmup_epochs=20, total_epochs=200):
    if epoch < warmup_epochs:
        return initial_lr * (epoch + 1) / warmup_epochs
    decay = (initial_lr - 1e-8) * (epoch - warmup_epochs) / (total_epochs - warmup_epochs)
    return max(initial_lr - decay, 1e-8)

# ─── TRANSFORMER DECODER BLOCK ───────────────────────────────────────────────
def transformer_decoder_block(query, key_value, head_size, num_heads, ff_dim, dropout=0.3):
    # 1) Self-attention + residual
    x = layers.LayerNormalization(epsilon=1e-6)(query)
    x1 = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(x, x)
    x1 = layers.Dropout(dropout)(x1)
    x = layers.Add()([x1, query])
    # 2) Cross-attention + residual
    x2 = layers.LayerNormalization(epsilon=1e-6)(x)
    x2 = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(x2, key_value)
    x2 = layers.Dropout(dropout)(x2)
    x = layers.Add()([x2, x])
    # 3) Feed-forward + residual
    x3 = layers.LayerNormalization(epsilon=1e-6)(x)
    x_ff = layers.Dense(ff_dim, activation='gelu')(x3)
    x_ff = layers.Dropout(dropout)(x_ff)
    x_ff = layers.Dense(query.shape[-1])(x_ff)
    return layers.Add()([x_ff, x])

# ─── BUILD THOSnet WITH TWO DECODERS ─────────────────────────────────────────
def build_thosnet(input_shape=(30,63)):
    left_in  = tf.keras.Input(shape=input_shape, name="left_hand")
    right_in = tf.keras.Input(shape=input_shape, name="right_hand")
    L = layers.Bidirectional(layers.LSTM(256, return_sequences=True))(left_in)
    R = layers.Bidirectional(layers.LSTM(256, return_sequences=True))(right_in)
    # Decoder A: left attends to right
    decA = transformer_decoder_block(L, R, head_size=256, num_heads=8, ff_dim=64)
    # Decoder B: right attends to left
    decB = transformer_decoder_block(R, L, head_size=256, num_heads=8, ff_dim=64)
    merged = layers.Concatenate()([decA, decB])
    flat   = layers.Flatten()(merged)
    x = layers.Dense(128, activation='gelu')(flat)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(64, activation='gelu')(x)
    x = layers.Dropout(0.3)(x)
    out = layers.Dense(9, activation='softmax', name="gesture_probs")(x)
    return tf.keras.Model([left_in, right_in], out)

# ─── LOAD & COMBINE ALL AUGMENTED DATA ──────────────────────────────────────
X1, y1 = np.load('processed_datasets/X_subject1_aug.npy'), np.load('processed_datasets/y_subject1_aug.npy')
X2, y2 = np.load('processed_datasets/X_subject2_aug.npy'), np.load('processed_datasets/y_subject2_aug.npy')
X3, y3 = np.load('processed_datasets/X_subject3_aug.npy'), np.load('processed_datasets/y_subject3_aug.npy')
X_all = np.vstack((X1, X2, X3))
y_all = np.vstack((y1, y2, y3))

# ─── SPLIT OFF A 10% TEST SET ─────────────────────────────────────────────────
X_train, X_test, y_train, y_test = train_test_split(
    X_all, y_all, test_size=0.10, random_state=42, stratify=y_all.argmax(axis=1)
)

# ─── SPLIT INTO LEFT/RIGHT HAND SEQUENCES ────────────────────────────────────
X_train_h1, X_train_h2 = X_train[:, :, :63], X_train[:, :, 63:]
X_test_h1,  X_test_h2  = X_test[:, :, :63],  X_test[:, :, 63:]

# ─── BUILD, COMPILE & CALLBACK ───────────────────────────────────────────────
model = build_thosnet((30,63))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
lr_callback = tf.keras.callbacks.LearningRateScheduler(
    lambda ep: lr_schedule(ep, initial_lr=3e-4, warmup_epochs=20, total_epochs=50),
    verbose=1
)

# ─── TRAIN ON 90% OF DATA ─────────────────────────────────────────────────────
history = model.fit(
    x=[X_train_h1, X_train_h2],
    y=y_train,
    epochs=50,
    batch_size=256,
    callbacks=[lr_callback],
    verbose=1
)

# ─── FINAL TRAIN LOSS & ACC ──────────────────────────────────────────────────
train_loss = history.history['loss'][-1]
train_acc  = history.history['accuracy'][-1]
print(f"\nFinal ▶️ Training Loss: {train_loss:.4f}, Accuracy: {train_acc:.4f}")

# ─── EVALUATE ON 10% HELD‐OUT SET ────────────────────────────────────────────
test_loss, test_acc = model.evaluate([X_test_h1, X_test_h2], y_test, verbose=0)
print(f"Final ▶️ Test     Loss: {test_loss:.4f}, Accuracy: {test_acc:.4f}\n")

# ─── PREDICT & CONFUSION MATRIX ──────────────────────────────────────────────
y_pred_probs = model.predict([X_test_h1, X_test_h2], verbose=0)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_test, axis=1)

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', 
            xticklabels=[f"G{i}" for i in range(9)],
            yticklabels=[f"G{i}" for i in range(9)])
plt.title("Confusion Matrix (Test Set)")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.tight_layout()
plt.show()

# ─── F1 SCORES PER CLASS & AVERAGE ───────────────────────────────────────────
f1_per_class = f1_score(y_true, y_pred, average=None)
avg_f1        = f1_score(y_true, y_pred, average='macro')
print("▶️ F1‐Score per Gesture:")
for i, f1 in enumerate(f1_per_class):
    print(f"   Gesture_{i}: {f1:.3f}")
print(f"\n▶️ Average (macro) F1‐Score: {avg_f1:.3f}\n")

# ─── LEARNING CURVES (LOSS & ACC) ─────────────────────────────────────────────
epochs = range(1, len(history.history['loss']) + 1)
plt.figure(figsize=(12,5))

plt.subplot(1,2,1)
plt.plot(epochs, history.history['loss'],  label='Train Loss')
plt.title("Training Loss")
plt.xlabel("Epoch"); plt.ylabel("Loss"); plt.legend()

plt.subplot(1,2,2)
plt.plot(epochs, history.history['accuracy'], label='Train Accuracy')
plt.title("Training Accuracy")
plt.xlabel("Epoch"); plt.ylabel("Accuracy"); plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
model.save_weights('models/thosnet_weights.h5')